# Assignment 3: Improving the Index

In this assignment, we will improve the search index and query functions from the previous assignment.

## Loading the Data and Defining Auxiliary Functions

This section is copied from the previous notebook.

In [1]:
import pickle, bz2, re
from collections import namedtuple, defaultdict, Counter
from IPython.display import display, HTML
from math import log10, sqrt

Summaries_file = 'data/emotion_Summaries.pkl.bz2'
Abstracts_file = 'data/emotion_Abstracts.pkl.bz2'

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )
for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )

In [2]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

In [3]:
def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

In [4]:
inverted_index = defaultdict(list)

for id in sorted(Summaries.keys()):
    term_set = set(preprocess(tokenize(Summaries[id].title)))
    if id in Abstracts:
        term_set.update(preprocess(tokenize(Abstracts[id])))
    for term in term_set:
        inverted_index[term].append(id)

## Stemming

As we could see from the results of the last assignment, our simple index doesn't handle punctuation and the difference between singular and plural versions of the same word very well. We won't go much into the details of tokenization and linguistic analysis here, because we also want to focus on scoring and ranking below. Therefore, we are using an existing library for tokenizatoin and stemming, namely the NLTK package. The following line will install NLTK if necessary (or you have to follow [these instructions](http://www.nltk.org/install.html) if that doesn't work):

In [5]:
! pip install --user nltk

In [6]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer
import nltk
nltk.download('punkt')
stemmer = EnglishStemmer()

s = '''Good muffins cost $3.88\nin New York.  Please buy me two of them.\n\nThanks.'''

print('INPUT TEXT:\n ', s)

print('TOKENIZE: ', tokenize(s))
print('WORD TOKENIZE: ', word_tokenize(s))

INPUT TEXT:
  Good muffins cost $3.88
in New York.  Please buy me two of them.

Thanks.
TOKENIZE:  ['Good', 'muffins', 'cost', '$3.88\nin', 'New', 'York.', '', 'Please', 'buy', 'me', 'two', 'of', 'them.\n\nThanks.']
WORD TOKENIZE:  ['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']


[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [7]:
print(stemmer.stem("processes"))

process


## Ranking

Another important method to improve our search results is to rank them, which can be done by calculating a score for each document based on the matching terms from the query. One such scoring method is *tf-idf*, which comes with several variants, as explained in the lecture slides.

In order to quickly calculate the scores for a term/document combination, we'll need quick access to a couple of things:

- tf(t,d): How often does a term occur in a document
- df(t): In how many documents does a term occur
- num_documents: The number of documents in our index

In [8]:
tf_matrix = defaultdict(Counter)

for doc_id in Summaries.keys():
    tokens = preprocess(tokenize(Summaries[doc_id].title))
    if (doc_id in Abstracts):
        tokens.extend(preprocess(tokenize(Abstracts[doc_id])))
    tf_matrix[doc_id] = Counter(tokens)

def tf(t,d):
    return float(tf_matrix[d][t])

def df(t):
    return float(len(inverted_index[t]))

num_documents = float(len(Summaries))

Let's test these functions with some examples:

In [9]:
print(tf('music', 33269144))
print(df('music'))
print(num_documents)

2.0
716.0
46483.0


With these helper functions, we can now easily calculate the _tf-idf_ weights of a term in a document by implementing the weighting formula from the slides, which you will do in the assignments below.

----------

# Tasks

**Your name:** Aldric de Jacquelin #2711498

### Task 1

Implement in the code block below the `smart_tokenize_and_preprocess` function using NLTK's functions for tokenization and stemming. 

In [10]:
# Smarter linguistic processing
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer

def smart_tokenize_and_preprocess(text):
    stemmer = EnglishStemmer()
    tokens = word_tokenize(text)
    preprocessed_tokens = [stemmer.stem(token.lower()) for token in tokens]
    return preprocessed_tokens

test1_text = "Books about Information Retrieval (IR) etc. cost at least $25.00!"
print(smart_tokenize_and_preprocess(test1_text))

['book', 'about', 'inform', 'retriev', '(', 'ir', ')', 'etc', '.', 'cost', 'at', 'least', '$', '25.00', '!']


Now we can make a smarter index based on this function. For practical purposes, the code below generates the smarter index on a subset of the data, as generating an index with stemming on the entire set would take too much time. (You don't need to change or add anything in the code block below. Just leave it as it is.)

In [11]:
# Below, we create our smarter index (based on a subset of the documents for demonstration purposes)
smarter_index = defaultdict(list)

# Here we define the subset (somewhat arbitrary):
subset_of_ids = list(key for key in Summaries.keys() if 33000000 <= key < 34000000)

# Building our smarter index:
for id in sorted(subset_of_ids):
    term_set = set(smart_tokenize_and_preprocess(Summaries[id].title))
    if id in Abstracts:
        term_set.update(smart_tokenize_and_preprocess(Abstracts[id]))
    for term in term_set:
        smarter_index[term].append(id)

Now implement the `smarter_and_query` function, based on the function `smarter_tokenize_and_preprocess` you defined above and accessing our new index `smarter_index`. You can start from copying the code for `and_query` from the last assignment. For that to work, you'll also have to copy the code for the `and_merge` function from the last assignment.

In [12]:
def merge_AND(list1, list2):
    ptr1 = 0
    ptr2 = 0
    result = []
    while ptr1 < len(list1) and ptr2 < len(list2):
        if list1[ptr1] == list2[ptr2]:
            result.append(list1[ptr1])
            ptr1 += 1
            ptr2 += 1
        elif list1[ptr1] < list2[ptr2]:
            ptr1 += 1
        else:
            ptr2 += 1
    return result

def smarter_and_query(query_string):
    tokens = smart_tokenize_and_preprocess(query_string)
    if not tokens:
        return []
    if tokens[0] in smarter_index:
        result_docs = list(set(smarter_index[tokens[0]]))
    else:
        return []

    for token in tokens[1:]:
        if token in smarter_index:
            result_docs = merge_AND(result_docs, list(set(smarter_index[token])))
        else:
            return []

    return result_docs

and_query_1 = "neural brain"
matched_1 = smarter_and_query(and_query_1)
and_query_2 = "neural network"
matched_2 = smarter_and_query(and_query_2)

display("and_query_1 = neural brain\n")
for doc_id in matched_1:
    display_summary(doc_id)

display("and_query_2 = neural network\n")
for doc_id in matched_2:
    display_summary(doc_id)

'and_query_1 = neural brain\n'

'and_query_2 = neural network\n'

### Task 2

Run the query "dance music Billboard" with the new `smarter_and_query` function from task 1. Does it return paper *33269144*? Explain what our new smarter function specifically contributes to the result (as compared to our previous naive implementations for tokenization and preprocessing)?

In [12]:
smarter_and_query("dance music Billboard")  

[]

**Answer:** 
The terms stemmed do not match any documents in the set indexed in smarter_index.
The subset of documents used to build the smarter_index might do not contain documents relevant to this query. The AND logic requires all terms to be present in a document. If even one term is missing in all documents, the result will be empty.

### Task 3

Now we move to a different subject and use our old index again. That is, we **don't** use the smarter functions defined above for tasks 3 to 5!

Create a function `tfidf(t,d)` that returns the tf-idf score of term `t` in document `d` by using `tf(t,d)`, `df(t)` and `num_documents` as defined above. To do this, first implement a function `idf(t)` to calculate the inverse document frequency, and then use this function to calculate the full tf-idf. Use the _add-one-smoothing_ version of idf, so we don't run into problems with terms that don't appear in the collection at all. The relevant formulas can be found on the lecture slides. Use tf-idf with plain (non-logarithmic) term frequency, as applied by scoring variant `ntn`. Test your function with the examples shown below. You can use the `log10(n)` function to calculate the base 10 logarithm.

Again, use our old (non-smart) index for this task and the tasks below, and **not** the functions defined in tasks 1 and 2.

In [15]:
from math import log10
# Your code here:
def idf(t):
    #inverse document frequency with add-one smoothing
    return log10((num_documents + 1) / (df(t) + 1))

def tfidf(t, d):
    #tf-idif score
    return tf(t, d) * idf(t)


print(tfidf('children', 33269144))
print(tfidf('music', 33269144))
print(tfidf('role', 33269144))

2.139216613177828
3.6235686736932524
0.7797987182790485


### Task 4

Create a function `query_ntn_nnn(query_string)`, which accepts as input a single query string of one or more words, and returns or prints a list of (up to) 10 best matching documents, along with their score. Use _tf-idf_ to calculate document scores based on the query, applying variant `ntn.nnn`, as above (see the formula for the `ntn.nnn` version of scoring on the lecture slides). Use an auxiliary function `score_ntn_nnn` to calculate the score. The results should be shown in descending order by score.

You can start by copying your functions `or_merge` and `or_query` from assignment 2, then expand that to rank the results, making use of the `tfidf(t,d)` function you created above.

Demonstrate your function by giving it the exemplary query string "effect of music and dance for young adults".

In [16]:
def merge_OR(list1, list2):
    ptr1 = 0 
    ptr2 = 0
    result = []
    while ptr1 < len(list1) and ptr2 < len(list2):
        if list1[ptr1] == list2[ptr2]:
            result.append(list1[ptr1])
            ptr1 += 1
            ptr2 += 1
        elif list1[ptr1] < list2[ptr2]:
            result.append(list1[ptr1])
            ptr1 += 1
        else:
            result.append(list2[ptr2])
            ptr2 += 1
    while ptr1 < len(list1):
        result.append(list1[ptr1])
        ptr1 += 1
    while ptr2 < len(list2):
        result.append(list2[ptr2])
        ptr2 += 1
    return result

def query_OR(query):
    tokens = preprocess(tokenize(query))
    result_docs = list(set(inverted_index[tokens[0]]))
    for token in tokens:
        if token in inverted_index:
            result_docs = merge_OR(result_docs, list(set(inverted_index[token])))
    return result_docs

def score_ntn_nnn(query_tokens, doc_id):
    score = sum(    tfidf(t, doc_id) 
                    for t in query_tokens 
                    if t in inverted_index)
    return score

def query_ntn_nnn(query_string) :
    query_tokens = preprocess(tokenize(query_string))
    result_docs = query_OR(query_string)
    scored_docs = [(doc_id, score_ntn_nnn(query_tokens, doc_id)) for doc_id in result_docs ]
    scored_docs.sort(key=lambda x:x[1], reverse=True)
    return scored_docs[:9 ]

# Example query
query_string = "effect of music and dance for young adults"
best_matches = query_ntn_nnn(query_string)
for doc_id, score in best_matches:
    print(f"docID: {doc_id}|score: {score}")
    display_summary(doc_id)

docID: 34094650|score: 39.74299733266672


docID: 34094650|score: 39.74299733266672


docID: 34094650|score: 39.74299733266672


docID: 24149889|score: 35.296019525665486


docID: 24149889|score: 35.296019525665486


docID: 24149889|score: 35.296019525665486


docID: 29621947|score: 31.47147833791991


docID: 29621947|score: 31.47147833791991


docID: 29621947|score: 31.47147833791991


### Task 5

In this last task, you should create a second version of the query function from Task 4, called `query_ntc_ntc`. This second version should use, as its name suggests, variant `ntc.ntc` instead of `ntn.nnn`, and therefore apply the cosine similarity measure, in addition to applying _tf-idf_. For this, consult the formula for variant `nnc.nnc` on the lecture slides and adopt it to include the _idf_ metric (that is, add the `t` element of `ntc`). (You can drop the square root of |q| in the formula, as indicated on the slides.)

As a first step, we can calculate beforehand the length of all document vectors (because they don't depend on the query) for document vectors consisting of _tf-idf_ values. The code below does just that, assuming that you defined the function `tfidf(t,d)` above (don't change this code block, just run it):

In [27]:
tfidf_length_values = defaultdict(int)

for doc_id in Summaries.keys():
    l = 0
    for t in tf_matrix[doc_id].keys():
        l += tfidf(t,doc_id) ** 2
    tfidf_length_values[doc_id] = sqrt(l)

def tfidf_length(d):
    return tfidf_length_values[d]

We can now get the length of a document vector by calling `tfidf_length(d)`.

Based on this, you can now implement `query_ntc_ntc` in the code block below. You should again first define an auxiliary function, called `score_ntc_ntc`. You can start by copy-pasting the code from Task 4.

To output the results, use the provided `display_summary` function to make the output a bit more like the results page of a search engine. Lastly, demonstrate your `query_ntc_ntc` function with the same example query as above: "effect of music and dance for young adults"

In [17]:
import numpy as np

def cosine_similarity(query_vector, document_vector):
    """cosine_similarity function to check for zero magnitude vectors & handle them. 
If either vector has zero magnitude, the cosine similarity should be set to 0, 
as there is no overlap in terms.
"""
    norm_query = np.linalg.norm(query_vector)
    norm_document = np.linalg.norm(document_vector)
    # Check for "zero magnitude
    if norm_query == 0 or norm_document == 0:
        return 0.0
    else:
        return np.dot(query_vector, document_vector) / (norm_query * norm_document)

def score_ntc_ntc(query_tokens, doc_id):
    all_terms = set(query_tokens) | set(tf_matrix[doc_id].keys())# craft a unified set of terms from both query and document
    query_vector = [tfidf(t, doc_id) if t in query_tokens else 0 for t in all_terms]# construct the query vector
    document_vector = [tfidf(t, doc_id) for t in all_terms]# create the document vector
    return cosine_similarity(query_vector, document_vector)# cosine similarity

def query_ntc_ntc(query_string):
    query_tokens = preprocess(tokenize(query_string)) 
    scored_docs = [(doc_id, score_ntc_ntc(query_tokens, doc_id)) for doc_id in Summaries.keys()]# scores for each document
    scored_docs.sort(key=lambda x: x[1], reverse=True)# sorting
    for doc_id, score in scored_docs[:9 ]:
        print(f"docID: {doc_id} | score: {score}")
        display_summary(doc_id)

# Example query:
query_ntc_ntc("effect of music and dance for young adults")


docID: 34094650 | score: 0.7552693397230237


docID: 26266024 | score: 0.7196124830915072


docID: 29238298 | score: 0.6777432716909281


docID: 22530296 | score: 0.6676443039363481


docID: 24568004 | score: 0.6440040678526759


docID: 25725908 | score: 0.632672495962279


docID: 24149889 | score: 0.6172513091846612


docID: 12748612 | score: 0.6166711989783237


docID: 21073970 | score: 0.6037797700838912


# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.